# Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
from read_data import read_file
from text_normalization import english_text_normalization, arabic_text_normalization
from text_processing import TextProcessing
import random
from tensorflow.keras.layers import LSTM, GRU, Attention, AdditiveAttention, MultiHeadAttention # type: ignore
from encoder_decoder_model import EncoderDecoderWithAttention, EncoderDecoderWithoutAttention
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
from model_check_point import check_point
from data_split import input_output_split, decoder_input_output
import warnings
import os
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
import re

# Ignore Warnings

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

# Set Early Stop For Encoder-Decoder Model

In [3]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Read Data

In [4]:
Corpus = read_file(file_path='Questions Answering Pairs.txt')
len(Corpus)

3653

In [5]:
random.seed(44)
Corpus = random.sample(Corpus, len(Corpus))

# Split The Data Into Encoder Input And Output

In [6]:
input_data, output_data = input_output_split(data=Corpus, text_normalization1=arabic_text_normalization, text_normalization2=english_text_normalization)

# Process The Data 

**Input Data**

In [7]:
text_processing_input = TextProcessing() # Input TextProcessing
all_words_input , words_input = text_processing_input.data_words(filter='"#$&*+/:=@[\\]^_`{|}~', data=input_data) # Set With Unique Words And Find Number Of Unique Words
words_to_index_input = text_processing_input.words_to_index_(words=words_input) # Convert Word To Integer Index
index_to_words_input = text_processing_input.index_to_word_(words=words_input)
input_sequences = text_processing_input.text_to_sequence(word_index=words_to_index_input, data=input_data) # Convert Text Into Squences Of Integer
input_data_max_length = len(max(input_sequences, key = len)) # Find Max Length
pad_input_data = text_processing_input.sequences_padding(input_sequence=input_sequences, max_length=input_data_max_length) # Zero Padding (Add Zeros To End Of Sequence In Input Sequences To Make All Sequences In Same Length)
word_count_input = text_processing_input.word_counts(data=input_data) # Number Of Occurrence Each Word

In [8]:
all_words_input

2464

**Output Data**

In [9]:
text_processing_output = TextProcessing() # Input TextProcessing
all_words_output , words_output = text_processing_output.data_words(filter='"#$&*+/=@[\\]^_`{|}~', data=output_data) # Set With Unique Words And Find Number Of Unique Words
words_to_index_output = text_processing_output.words_to_index_(words=words_output) # Conver Word To Integer Index
index_to_words_output = text_processing_output.index_to_word_(words=words_output)
output_sequences = text_processing_output.text_to_sequence(word_index=words_to_index_output, data=output_data) # Convert Text Into Squences Of Integer
word_count_output = text_processing_output.word_counts(data=output_data) # Number Of Occurrence Each Word

In [10]:
all_words_output

7956

# Generate Decoder Input And Output

In [11]:
decoder_input, decoder_output = decoder_input_output(output_data= output_sequences)

# Find Decoder Max Length

In [12]:
decoder_max_length = len(max(decoder_input, key = len)) # Find Max Length Of Decoder
decoder_max_length

104

# Padding Decoder Data

In [13]:
pad_decoder_input = text_processing_output.sequences_padding(input_sequence=decoder_input, max_length=decoder_max_length)
pad_decoder_output = text_processing_output.sequences_padding(input_sequence=decoder_output, max_length=decoder_max_length)

# Encoder-Decoder Without Attention

**LSTM**

In [14]:
lstm_callback_1 = check_point('lstm_1')
model_encoder_lstm_1 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_lstm_1 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
encoder_decoder_lstm_1 = EncoderDecoderWithoutAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm_1, model_decoder = model_decoder_lstm_1, unit = all_words_output, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=lstm_callback_1, early_stop=early_stopping)
encoder_inputs, encoder_states = encoder_decoder_lstm_1.encoder()
decoder_inputs, decoder_outputs = encoder_decoder_lstm_1.decoder(encoder_states=encoder_states)
model_lstm_1 = encoder_decoder_lstm_1.build_model()
encoder_decoder_lstm_1.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_1, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_lstm_1.model_fit(model = model_lstm_1, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
115/115 [==============================] - 26s 205ms/step - loss: 6.6062 - sparse_categorical_accuracy: 0.0452
Epoch 2/100
115/115 [==============================] - 23s 198ms/step - loss: 5.9172 - sparse_categorical_accuracy: 0.0516
Epoch 3/100
115/115 [==============================] - 22s 194ms/step - loss: 5.6169 - sparse_categorical_accuracy: 0.0773
Epoch 4/100
115/115 [==============================] - 22s 192ms/step - loss: 5.2956 - sparse_categorical_accuracy: 0.1098
Epoch 5/100
115/115 [==============================] - 23s 197ms/step - loss: 4.9433 - sparse_categorical_accuracy: 0.1729
Epoch 6/100
115/115 [==============================] - 23s 198ms/step - loss: 4.5497 - sparse_categorical_accuracy: 0.2613
Epoch 7/100
115/115 [==============================] - 23s 198ms/step - loss: 4.1893 - sparse_categorical_accuracy: 0.3446
Epoch 8/100
115/115 [==============================] - 24s 205ms/step - loss: 3.9023 - sparse_categorical_accuracy: 0.3956
Epoch 9/100
115/

**GRU**

In [15]:
gru_callback_1 = check_point('gru_1')
model_encoder_gru_1 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_gru_1 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
encoder_decoder_2 = EncoderDecoderWithoutAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru_1, model_decoder = model_decoder_gru_1, unit = all_words_output, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=gru_callback_1, early_stop=early_stopping)
encoder_inputs, encoder_states = encoder_decoder_2.encoder()
decoder_inputs, decoder_outputs = encoder_decoder_2.decoder(encoder_states=encoder_states)
model_gru_1 = encoder_decoder_2.build_model()
encoder_decoder_2.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_1, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_2.model_fit(model = model_gru_1, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
115/115 [==============================] - 23s 177ms/step - loss: 6.6521 - sparse_categorical_accuracy: 0.0519
Epoch 2/100
115/115 [==============================] - 21s 184ms/step - loss: 5.9633 - sparse_categorical_accuracy: 0.0518
Epoch 3/100
115/115 [==============================] - 22s 187ms/step - loss: 5.8751 - sparse_categorical_accuracy: 0.0544
Epoch 4/100
115/115 [==============================] - 22s 189ms/step - loss: 5.7028 - sparse_categorical_accuracy: 0.0768
Epoch 5/100
115/115 [==============================] - 22s 189ms/step - loss: 5.1427 - sparse_categorical_accuracy: 0.1515
Epoch 6/100
115/115 [==============================] - 21s 185ms/step - loss: 4.4848 - sparse_categorical_accuracy: 0.2787
Epoch 7/100
115/115 [==============================] - 21s 185ms/step - loss: 3.9473 - sparse_categorical_accuracy: 0.3906
Epoch 8/100
115/115 [==============================] - 23s 203ms/step - loss: 3.6062 - sparse_categorical_accuracy: 0.4397
Epoch 9/100
115/

# Encoder-Decoder With Attention

**1-Luong Attension**

*LSTM*

In [16]:
lstm_callback_2 = check_point('lstm_2')
model_encoder_lstm_2 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_lstm_2 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = Attention(use_scale=True)
encoder_decoder_3 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm_2, model_decoder = model_decoder_lstm_2, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=lstm_callback_2, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_3.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_3.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_lstm_2 = encoder_decoder_3.build_model()
encoder_decoder_3.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_2, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_3.model_fit(model = model_lstm_2, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100


W0000 00:00:1724791952.457621       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


115/115 [==============================] - 30s 248ms/step - loss: 4.0356 - sparse_categorical_accuracy: 0.5243
Epoch 2/100
115/115 [==============================] - 30s 260ms/step - loss: 2.8300 - sparse_categorical_accuracy: 0.5472
Epoch 3/100
115/115 [==============================] - 29s 252ms/step - loss: 2.7221 - sparse_categorical_accuracy: 0.5618
Epoch 4/100
115/115 [==============================] - 28s 244ms/step - loss: 2.5693 - sparse_categorical_accuracy: 0.5819
Epoch 5/100
115/115 [==============================] - 28s 243ms/step - loss: 2.3611 - sparse_categorical_accuracy: 0.6179
Epoch 6/100
115/115 [==============================] - 28s 244ms/step - loss: 2.1682 - sparse_categorical_accuracy: 0.6612
Epoch 7/100
115/115 [==============================] - 28s 243ms/step - loss: 2.0118 - sparse_categorical_accuracy: 0.6880
Epoch 8/100
115/115 [==============================] - 29s 256ms/step - loss: 1.8806 - sparse_categorical_accuracy: 0.7084
Epoch 9/100
115/115 [=======

*GRU*

In [17]:
gru_callback_2 = check_point('gru_2')
model_encoder_gru_2 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_gru_2 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = Attention(use_scale=True)
encoder_decoder_4 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru_2, model_decoder = model_decoder_gru_2, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=gru_callback_2, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_4.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_4.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_gru_2 = encoder_decoder_4.build_model()
encoder_decoder_4.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_2, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_4.model_fit(model = model_gru_2, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100


W0000 00:00:1724794848.162810       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


115/115 [==============================] - 31s 255ms/step - loss: 4.1722 - sparse_categorical_accuracy: 0.5281
Epoch 2/100
115/115 [==============================] - 30s 257ms/step - loss: 2.8554 - sparse_categorical_accuracy: 0.5477
Epoch 3/100
115/115 [==============================] - 28s 242ms/step - loss: 2.7283 - sparse_categorical_accuracy: 0.5615
Epoch 4/100
115/115 [==============================] - 28s 241ms/step - loss: 2.5242 - sparse_categorical_accuracy: 0.5872
Epoch 5/100
115/115 [==============================] - 30s 258ms/step - loss: 2.2880 - sparse_categorical_accuracy: 0.6306
Epoch 6/100
115/115 [==============================] - 28s 245ms/step - loss: 2.0303 - sparse_categorical_accuracy: 0.6802
Epoch 7/100
115/115 [==============================] - 33s 289ms/step - loss: 1.8131 - sparse_categorical_accuracy: 0.7191
Epoch 8/100
115/115 [==============================] - 32s 281ms/step - loss: 1.6597 - sparse_categorical_accuracy: 0.7384
Epoch 9/100
115/115 [=======

**2-Bahdanau Attension**

*LSTM*

In [18]:
lstm_callback_3 = check_point('lstm_3')
model_encoder_lstm_3 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_lstm_ = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = AdditiveAttention(use_scale=True)
encoder_decoder_5 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm_3, model_decoder = model_encoder_lstm_3, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=lstm_callback_3, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_5.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_5.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_lstm_3 = encoder_decoder_5.build_model()
encoder_decoder_5.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_3, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_5.model_fit(model = model_lstm_3, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100


W0000 00:00:1724797637.008761       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


115/115 [==============================] - 30s 250ms/step - loss: 4.2917 - sparse_categorical_accuracy: 0.5276
Epoch 2/100
115/115 [==============================] - 29s 250ms/step - loss: 2.8919 - sparse_categorical_accuracy: 0.5377
Epoch 3/100
115/115 [==============================] - 29s 252ms/step - loss: 2.6769 - sparse_categorical_accuracy: 0.5624
Epoch 4/100
115/115 [==============================] - 29s 253ms/step - loss: 2.4750 - sparse_categorical_accuracy: 0.5858
Epoch 5/100
115/115 [==============================] - 30s 258ms/step - loss: 2.2856 - sparse_categorical_accuracy: 0.6150
Epoch 6/100
115/115 [==============================] - 29s 253ms/step - loss: 2.1079 - sparse_categorical_accuracy: 0.6520
Epoch 7/100
115/115 [==============================] - 29s 254ms/step - loss: 1.9544 - sparse_categorical_accuracy: 0.6893
Epoch 8/100
115/115 [==============================] - 29s 252ms/step - loss: 1.8324 - sparse_categorical_accuracy: 0.7140
Epoch 9/100
115/115 [=======

*GRU*

In [19]:
gru_callback_3 = check_point('gru_3')
model_encoder_gru_3 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_gru_3 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = Attention(use_scale=True)
encoder_decoder_6 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru_3, model_decoder = model_decoder_gru_3, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=gru_callback_3, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_6.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_6.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_gru_3 = encoder_decoder_6.build_model()
encoder_decoder_6.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_3, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_6.model_fit(model = model_gru_3, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100


W0000 00:00:1724800661.911467       1 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


115/115 [==============================] - 27s 227ms/step - loss: 4.0805 - sparse_categorical_accuracy: 0.5342
Epoch 2/100
115/115 [==============================] - 27s 232ms/step - loss: 2.8325 - sparse_categorical_accuracy: 0.5550
Epoch 3/100
115/115 [==============================] - 26s 230ms/step - loss: 2.7220 - sparse_categorical_accuracy: 0.5652
Epoch 4/100
115/115 [==============================] - 26s 229ms/step - loss: 2.5354 - sparse_categorical_accuracy: 0.5908
Epoch 5/100
115/115 [==============================] - 28s 246ms/step - loss: 2.2820 - sparse_categorical_accuracy: 0.6368
Epoch 6/100
115/115 [==============================] - 27s 234ms/step - loss: 2.0423 - sparse_categorical_accuracy: 0.6886
Epoch 7/100
115/115 [==============================] - 27s 231ms/step - loss: 1.8562 - sparse_categorical_accuracy: 0.7156
Epoch 8/100
115/115 [==============================] - 27s 234ms/step - loss: 1.7122 - sparse_categorical_accuracy: 0.7336
Epoch 9/100
115/115 [=======

**3-MultiHead Attesnion**

*LSTM*

In [20]:
lstm_callback_4 = check_point('lstm_4')
model_encoder_lstm_4 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_lstm_4 = LSTM(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
encoder_decoder_7 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_lstm_4, model_decoder = model_decoder_lstm_4, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=lstm_callback_4, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_7.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_7.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_lstm_4 = encoder_decoder_7.build_model()
encoder_decoder_7.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_lstm_4, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_7.model_fit(model = model_lstm_4, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
115/115 [==============================] - 29s 238ms/step - loss: 3.9475 - sparse_categorical_accuracy: 0.5293
Epoch 2/100
115/115 [==============================] - 27s 238ms/step - loss: 2.8016 - sparse_categorical_accuracy: 0.5487
Epoch 3/100
115/115 [==============================] - 28s 242ms/step - loss: 2.6550 - sparse_categorical_accuracy: 0.5614
Epoch 4/100
115/115 [==============================] - 28s 239ms/step - loss: 2.4423 - sparse_categorical_accuracy: 0.5867
Epoch 5/100
115/115 [==============================] - 27s 237ms/step - loss: 2.2084 - sparse_categorical_accuracy: 0.6403
Epoch 6/100
115/115 [==============================] - 27s 234ms/step - loss: 1.9703 - sparse_categorical_accuracy: 0.6859
Epoch 7/100
115/115 [==============================] - 27s 235ms/step - loss: 1.7996 - sparse_categorical_accuracy: 0.7107
Epoch 8/100
115/115 [==============================] - 27s 235ms/step - loss: 1.6744 - sparse_categorical_accuracy: 0.7282
Epoch 9/100
115/

*GRU*

In [21]:
gru_callback_4 = check_point('gru_4')
model_encoder_gru_4 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
model_decoder_gru_4 = GRU(units=128, return_sequences=True, return_state=True, seed = 33)
attention_layer = MultiHeadAttention(num_heads=4, key_dim=32)
encoder_decoder_8 = EncoderDecoderWithAttention(input_dim_encoder = all_words_input, input_dim_decoder = all_words_output , output_dim_encoder = 100, output_dim_decoder = 100, input_length_encoder = input_data_max_length , input_length_decoder = decoder_max_length, model_encoder = model_encoder_gru_4, model_decoder = model_decoder_gru_4, unit = all_words_output, attention_layer = attention_layer, max_length_input=input_data_max_length, max_length_output=decoder_max_length, callback=gru_callback_4, early_stop=early_stopping)
output_data, encoder_inputs, encoder_states = encoder_decoder_8.encoder_()
decoder_inputs, decoder_outputs = encoder_decoder_8.decoder_(encoder_states=encoder_states, encoder_outputs=output_data)
model_gru_4 = encoder_decoder_8.build_model()
encoder_decoder_8.model_compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', model = model_gru_4, metrics=['sparse_categorical_accuracy'])
history = encoder_decoder_8.model_fit(model = model_gru_4, epochs = 100, batch_size = 32, encoder_input = pad_input_data, decoder_input = pad_decoder_input, decoder_output = pad_decoder_output)

Epoch 1/100
115/115 [==============================] - 28s 227ms/step - loss: 3.8082 - sparse_categorical_accuracy: 0.5355
Epoch 2/100
115/115 [==============================] - 27s 234ms/step - loss: 2.7243 - sparse_categorical_accuracy: 0.5611
Epoch 3/100
115/115 [==============================] - 27s 233ms/step - loss: 2.5271 - sparse_categorical_accuracy: 0.5823
Epoch 4/100
115/115 [==============================] - 27s 234ms/step - loss: 2.2746 - sparse_categorical_accuracy: 0.6278
Epoch 5/100
115/115 [==============================] - 27s 233ms/step - loss: 2.0056 - sparse_categorical_accuracy: 0.6852
Epoch 6/100
115/115 [==============================] - 27s 233ms/step - loss: 1.7928 - sparse_categorical_accuracy: 0.7187
Epoch 7/100
115/115 [==============================] - 27s 233ms/step - loss: 1.6496 - sparse_categorical_accuracy: 0.7355
Epoch 8/100
115/115 [==============================] - 27s 231ms/step - loss: 1.5455 - sparse_categorical_accuracy: 0.7478
Epoch 9/100
115/